In [1]:
import pynq
from dishsoap import CountStreamTest, TestDmaStreamOverlay

ol = pynq.Overlay('test_dma_stream.bit', download=True)

In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=1, width=80, depth=3)
#pp.pprint(ol.ip_dict['axi_dma_0'])
#pp.pprint(ol.ip_dict['count_stream_test_0'])

In [19]:
from pynq import allocate # for DMA interactions

def chan_status(chan):
    if chan is None:
        return ''
    
    r = chan.running
    i = chan.idle
    e = chan.error

    return f'Run:{r} Idle:{i} Err:{e}'

def dump_regs(dma, offset):
    info = ''
    cr = dma.mmio.read(offset + 0x0)
    sr = dma.mmio.read(offset + 0x4)
    lo = dma.mmio.read(offset + 0x18)
    hi = dma.mmio.read(offset + 0x1c)
    sz = dma.mmio.read(offset + 0x28)
    info += f'CR:   0x{cr:08x}\n'
    info += f'SR:   0x{sr:08x}\n'
    info += f'addr: 0x{hi:08x}{lo:08x}\n'
    info += f'len:  0x{sz:08x}\n'
    return info

def dump_stream_regs(count_stream):
    info = ''
    lo    = count_stream.read(0x0)
    hi    = count_stream.read(0x4)
    info += f'StreamCfg: {lo} to {hi}\n'
    debug = count_stream.read(0xc)
    info += f'TVALID: {(debug & (1 << 0)) >> 0} '
    info += f'TLAST: {(debug & (1 << 1)) >> 1} '
    info += f'TREADY: {(debug & (1 << 2)) >> 2}\n'
    info += f'counter_start: {(debug & (1 << 4)) >> 4}\n'
    info += f'tx_done: {(debug & (1 << 8)) >> 8} '
    info += f'tx_en: {(debug & (1 << 9)) >> 9}\n'
    info += f'read_pointer: {debug >> 16}\n'
    return info


def debug_dma(dma):
    if dma.sendchannel is not None:
        print('Send Channel', chan_status(dma.sendchannel))
        print(dump_regs(dma, 0x0))
    if dma.recvchannel is not None:
        print('Recv Channel', chan_status(dma.recvchannel))
        print(dump_regs(dma, 0x30))

def debug_stream(count_stream):
    print(dump_stream_regs(count_stream))
    
def debug_both(dma, count_stream):
    debug_dma(dma)
    debug_stream(count_stream)
    

In [4]:
count = 20
start = 10

debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
dma_result = allocate(shape=(32,), dtype='u4')
print(dma_result)
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
ol.axi_dma_0.recvchannel.transfer(dma_result, nbytes=count*4)
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
ol.count_stream_test_0.write(0x0, start)         # counter_lo
ol.count_stream_test_0.write(0x4, start+count-1) # counter_hi
ol.count_stream_test_0.write(0x8, 1)             # counter start
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
ol.axi_dma_0.recvchannel.wait()
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
print(dma_result)

Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00000000
addr: 0x0000000000000000
len:  0x00000000

StreamCfg: 0 to 0, debug:00000104
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00000000
addr: 0x0000000000000000
len:  0x00000000

StreamCfg: 0 to 0, debug:00000104
Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00000000
addr: 0x0000000016849000
len:  0x00000050

StreamCfg: 0 to 0, debug:00000104
Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00000000
addr: 0x0000000016849000
len:  0x00000050

StreamCfg: 10 to 29, debug:00010104


KeyboardInterrupt: 

In [20]:
count = 5
start = 200

dma_result2 = allocate(shape=(count,), dtype='u4')
print(dma_result2)
debug_stream(ol.count_stream_test_0)
ol.count_stream_test_0.write(0x0, start)         # counter_lo
ol.count_stream_test_0.write(0x4, start+count-1) # counter_hi
ol.count_stream_test_0.write(0x8, 1)             # counter start
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
ol.axi_dma_0.recvchannel.transfer(dma_result2)
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)
ol.axi_dma_0.recvchannel.wait()
print(dma_result2)
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)

[0 0 0 0 0]
StreamCfg: 200 to 204
TVALID: 0 TLAST: 0 TREADY: 4
counter_start: 0
tx_done: 256 tx_en: 0
read_pointer: 5

Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00001000
addr: 0x000000001684d000
len:  0x00000014

StreamCfg: 200 to 204
TVALID: 0 TLAST: 0 TREADY: 4
counter_start: 0
tx_done: 256 tx_en: 0
read_pointer: 5



RuntimeError: DMA channel not idle

In [14]:
debug_dma(ol.axi_dma_0)
debug_stream(ol.count_stream_test_0)

Recv Channel Run:True Idle:False Err:False
CR:   0x00011003
SR:   0x00001000
addr: 0x000000001684d000
len:  0x00000014

StreamCfg: 200 to 204, debug:00050104


In [ ]:
dir(ol.axi_dma_0.recvchannel)

### Notes

I think we need to enable interrupts on the DMA. Other things seem to be working (like the bit clearing in register 0x8)

Might wanna test the other pieces of count_stream_test_0 as well.